<a href="https://colab.research.google.com/github/pberiaintelle/Coviability-indicators/blob/main/Landsat_ts_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adapted from https://colab.research.google.com/drive/1bo0t-Oodq2K68t_v42DoNg8OpvSq_Nr0#scrollTo=5tgEyLSrA7tL

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qPL-TWGgYUhW76onlocf3UahBDUMiaVvQR22Mxb8oFQ&tc=nG5ieOda7Ue6BWEKd4qpyEyQJA3nukg8C9x3Rr-QDxc&cc=AqVnhf27PawM_vod6bOCPz5xaqkO-hRKB4w9XWybK7Y

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhXS9iYzmkDa75v2QLtzVu2St6Xl_G2z5PTsH9gJPmNu0lPivAnh2NA

Successfully saved authorization token.


In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import folium


# Region reduction function

# Reduction arguments
Define a global dictionary that provides arguments to the Earth Engine reduceRegion image method.

In [ ]:
reReArgs = {
  'reducer': ee.Reducer.mean(),
  'geometry': ee.Geometry.Point([0, 0]),
  'scale': 1000,
  'crs': 'EPSG:5070',
  'bestEffort': True,
  'maxPixels': 1e14,
  'tileScale': 4}

In [ ]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4618',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

# Feature collection into dictionary

In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

In [ ]:
#Example
pdsi_dict = fc_to_dict(pdsi_stat_fc).getInfo()

NameError: ignored

In [ ]:
#print small part
print(type(pdsi_dict), '\n')
for prop in pdsi_dict.keys():
    print(prop + ':', pdsi_dict[prop][0:3] + ['...'])

# Convert the Python dictionary to a pandas DataFrame.

In [ ]:
pdsi_df = pd.DataFrame(pdsi_dict)
#Display
display(pdsi_df)
print(pdsi_df.dtypes)

Add date column

In [ ]:
# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

# Reduction function
This function reduces the pixels intersecting a region to a statistic. It operates on a single ee.Image and returns an ee.Feature (after the reduction, the pixel values and geometry matter not, so just return an aspatial feature). The plan is to map this over an ee.ImageCollection.

Derive a series of date properties that will be used in charting.
Apply the reduceRegion method to the image using arguments provided by the global reReArgs dictionary.
Return an ee.Feature without geometry that contains the region reduction statistic result (an ee.dictionary), all image properties, and the derived date properties.

In [ ]:
def regionReduce(img):
  eeDate = img.date()
  year = eeDate.get('year')
  month = eeDate.getRelative('month', 'year')
  doy = eeDate.getRelative('day', 'year')
  date = eeDate.format('YYYY-MM-dd')

  stat = img.reduceRegion(
    reducer = reReArgs['reducer'],
    geometry = reReArgs['geometry'],
    scale = reReArgs['scale'],
    crs = reReArgs['crs'],
    bestEffort = reReArgs['bestEffort'],
    maxPixels = reReArgs['maxPixels'],
    tileScale = reReArgs['tileScale'])

  return(ee.Feature(None, stat) \
    .copyProperties(img, img.propertyNames())
    .set({
      'DOY': doy,
      'Month': month,
      'Year': year,
      'Date': date}))

Reduction to list

In [ ]:
def getReReList(col, props):
  dict = col.map(regionReduce) \
    .filter(ee.Filter.notNull(props)) \
    .reduceColumns(
      reducer = ee.Reducer.toList().repeat(len(props)),
      selectors = props)

  return(ee.List(dict.get('list')).getInfo())

# Prepare Landsat time series

In [ ]:
start_day = 350
end_day = 362

#latitude = -35.030
#longitude = -6.70

Create polygon from coordinates

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

lat_point_list = [-6.369, -6.369,-7.709, -7.709]
lon_point_list = [-34.740,-35.633, -35.633,-34.740]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4674', geometry=[polygon_geom])

polygon = polygon.to_file(filename='polygon.geojson', driver='GeoJSON')
#polygon.to_file(filename='polygon.shp', driver="ESRI Shapefile")


Download it to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Make lat. and long. vars an `ee.Geometry.Point`.
aoi = ee.Geometry.Rectangle([-34.740, -6.369, -35.633, -7.709])
#point = ee.Geometry.Point([longitude, latitude])

# Define a function to get and rename bands of interest from OLI.
def rename_oli(img):
  return (img.select(
      ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']),
      ee.List(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])))

# Define a function to get and rename bands of interest from ETM+.
def rename_etm(img):
  return (img.select(
      ee.List(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']),
      ee.List(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])))

# Define a function to mask out clouds and cloud shadows.
def cfmask(img):
  cloud_shadow_bi_mask = 1 << 3
  cloud_bit_mask = 1 << 5
  qa = img.select('pixel_qa')
  mask = qa.bitwiseAnd(cloud_shadow_bi_mask).eq(0).And(
      qa.bitwiseAnd(cloud_bit_mask).eq(0))
  return img.updateMask(mask)

# Define a function to add year as an image property.
def set_year(img):
  year = ee.Image(img).date().get('year')
  return img.set('Year', year)

# Define a function to calculate NBR.
def calc_nbr(img):
  return img.normalizedDifference(ee.List(['NIR', 'Red'])).rename('NDVI')
#Define function to clip image
def clip(img, aoi):
  clipped_img= mask(img, aoi, crop=True)
  return clipped_img

# Define a function to prepare OLI images.
def prep_oli(img):
  orig = img
  img = img.clip(ee.Geometry.Rectangle([-34.740, -6.369, -35.633, -7.709]))
  img = rename_oli(img)
  img = cfmask(img)
  img = calc_nbr(img)
  img = img.copyProperties(orig, orig.propertyNames())
  return set_year(img)

# Define a function to prepare TM/ETM+ images.
def prep_etm(img):
  orig = img
  img = img.clip(ee.Geometry.Rectangle([-34.740, -6.369, -35.633, -7.709]))
  img = rename_etm(img)
  img = cfmask(img)
  img = calc_nbr(img)
  img = img.copyProperties(orig, orig.propertyNames())
  return set_year(img)

# Import image collections for each Landsat sensor (surface reflectance).
tm_col = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
etm_col = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
oli_col = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')#.filterMetadata('CLOUD_COVER', 'less_than',20)#.filterDate('2020-01-01', '2020-12-31')

# Filter collections and prepare them for merging.
oli_col = oli_col.filterBounds(aoi).filter(
    ee.Filter.calendarRange(start_day, end_day, 'day_of_year')).map(prep_oli)

etm_col = etm_col.filterBounds(aoi).filter(
    ee.Filter.calendarRange(start_day, end_day, 'day_of_year')).map(prep_etm)

tm_col = tm_col.filterBounds(aoi).filter(
    ee.Filter.calendarRange(start_day, end_day, 'day_of_year')).map(prep_etm)

# Merge the collections.
landsat_col = oli_col.merge(etm_col).merge(tm_col)

# Get a distinct year collection.
distinct_year_col = landsat_col.distinct('Year')

# Define a filter that identifies which images from the complete collection
# match the year from the distinct year collection.
join_filter = ee.Filter.equals(leftField='Year', rightField='Year')

# Define a join.
join = ee.Join.saveAll('year_matches')

# Apply the join and convert the resulting FeatureCollection to an
# ImageCollection.
join_col = ee.ImageCollection(
    join.apply(distinct_year_col, landsat_col, join_filter))


In [ ]:
print(join_col.size().getInfo())

5


# Reduce data per year

In [ ]:

# Define a function to apply mean reduction among matching year collections.
def reduce_by_join(img):
  year_col = ee.ImageCollection.fromImages(ee.Image(img).get('year_matches'))
  return year_col.reduce(ee.Reducer.mean()).rename('NDVI').set(
      'system:time_start',
      ee.Image(img).date().update(month=8, day=1).millis())

# Apply the `reduce_by_join` function to the list of annual images in the
# properties of the join collection.
landsaty = join_col.map(reduce_by_join)

In [ ]:
print(landsaty.size().getInfo())

5


In [ ]:
landsaty_NDVI = landsaty.select("NDVI").median().set("system:time_start", ee.Date("2022"))

In [ ]:
print(landsaty_NDVI.size().getInfo())

AttributeError: ignored

Reduce the region to visualize years

In [ ]:
point = ee.Geometry.Point(-6.70, -35.030)


In [ ]:
# reset global args
reReArgs['reducer'] = ee.Reducer.first()
reReArgs['scale'] = 30
reReArgs['geometry'] = aoi
reReArgs['crs'] = 'EPSG:4618'

# Get a list containing a series of lists, one for each property given.
pointNbrList = getReReList(tm_col, ['Year', 'Date', 'NDVI', 'dtype'])

In [ ]:
#Define data structure
#cols = {'Year': int, 'Month': int, 'DOY': int, 'Date': str, 'NDVI': float}

def eeList2Df(list, cols):
  df = pd.DataFrame(list).transpose()
  df.columns = [k for k in cols.keys()]
  return(df.astype(cols))

Transform and tidy the table.

In [ ]:
#cols = {'DOY': int, 'Year': int, 'Date': str, 'NDVI': float}
cols = {'Year': int, 'Date': str, 'NDVI': float, 'dtype': object}
pointNbrDf = eeList2Df(pointNbrList, cols)
pointNbrDf.tail(5)

,Year,Date,NDVI,dtype


In [ ]:
!pip install geemap
import geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.8 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=6aa9d8c8126b9605d6879b253bf65a04406bf9e688ad9b36ebd276e126f89188
  Stor

In [ ]:
#Visualize image
Map = geemap.Map()
# Load an image.

# Define the visualization parameters.
ndviViz = {'min': 0.0, 'max': 1, 'palette': ['yellow', 'green']}
# Center the map and display the image.
Map.setCenter(-35,-7, 9) # Paraiba coast
Map.addLayer(landsaty, ndviViz, 'reduced per year', False)
Map.addLayer(join_col, ndviViz, 'NDVI', False)
Map.addLayer(landsaty_NDVI, ndviViz, 'imageNDVI', False)
# Display the map
Map

Map(center=[-7, -35], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

# Download collection

In [ ]:
!pip install wxee

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 7.2 MB/s eta 0:00:00


In [ ]:
import wxee
wxee.Initialize()

In [ ]:
files = landsaty_NDVI.wx.to_tif(
    out_dir="C:/Users/pablo/Documents/IRD/data/Landsat_ts_colab",
    #prefix="wx_",
    region=aoi,
    scale=30,
    crs='EPSG:4618'
)

files

EEException: ignored

In [ ]:
# Export the image to Drive
task = ee.batch.Export.image.toDrive(**{
  'image': landsaty_NDVI,
  'description': 'landsat_medianDOY358_362',
  'folder': 'Paraiba',
  'scale': 30,
  'region': aoi,
  'maxPixels': 10000000000
})
task.start()

In [ ]:
# Export the image to an Earth Engine asset.
task = ee.batch.Export.image.toAsset(**{
  'image': landsaty_NDVI,
  'description': 'landsat median',
  'assetId': 'users/pberiain5/exampleExport',
  'scale': 30,
  'region': geometry.getInfo()['coordinates']
})
task.start()

# Prepare dataframe

In [ ]:
reduce_landsat = create_reduce_region_function(
    geometry=point, reducer=ee.Reducer.first(), scale=30, crs='EPSG:4618')

nbr_stat_fc = ee.FeatureCollection(landsat_col.map(reduce_landsat)).filter(
    ee.Filter.notNull(landsat_col.first().bandNames()))

In [ ]:
2023-1984

39

In [ ]:
nbr_stat_fc

# FEATURE COLLECTION TO DICTIONARY

In [ ]:
#Example
nbr_dict = fc_to_dict(nbr_stat_fc).getInfo()

In [ ]:
#print small part
print(type(nbr_dict), '\n')
for prop in nbr_dict.keys():
    print(prop + ':', nbr_dict[prop][0:3] + ['...'])

<class 'dict'> 

NDVI: [0.6600085496902466, 0.6588217616081238, 0.8593696355819702, '...']
millis: [1375360327810, 1406896201573, 1438432123441, '...']
system:index: ['1_1_LC08_214064_20130325', '1_1_LC08_214064_20140104', '1_1_LC08_214064_20150107', '...']


Preview dataframe

In [ ]:
nbr_df = pd.DataFrame(nbr_dict)
display(nbr_df.head(5))
#print(nbr_df.dtypes)


,NDVI,millis,system:index
0,0.660009,1375360327810,1_1_LC08_214064_20130325
1,0.658822,1406896201573,1_1_LC08_214064_20140104
2,0.859370,1438432123441,1_1_LC08_214064_20150107
3,0.410217,1470054518850,1_1_LC08_214064_20160211
4,0.763187,1501590511460,1_1_LC08_214064_20170301


In [ ]:
# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [ ]:
nbr_df = add_date_info(nbr_df)
nbr_df.head(5)

,NDVI,millis,system:index,Timestamp,Year,Month,Day,DOY
0,0.660009,1375360327810,1_1_LC08_214064_20130325,2013-08-01 12:32:07.810,2013,8,1,213
1,0.658822,1406896201573,1_1_LC08_214064_20140104,2014-08-01 12:30:01.573,2014,8,1,213
2,0.859370,1438432123441,1_1_LC08_214064_20150107,2015-08-01 12:28:43.441,2015,8,1,213
3,0.410217,1470054518850,1_1_LC08_214064_20160211,2016-08-01 12:28:38.850,2016,8,1,214
4,0.763187,1501590511460,1_1_LC08_214064_20170301,2017-08-01 12:28:31.460,2017,8,1,213


# Calendar heatmap

In [ ]:
alt.Chart(nbr_df).mark_rect().encode(
    x='Year:O',
    y='Month:O',
    color=alt.Color(
        'mean(NDVI):Q', scale=alt.Scale(scheme='yellowgreen', domain=(0, 1))),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Month:O', title='Month'),
        alt.Tooltip('mean(NBR):Q', title='NBR')
    ]).properties(width=600, height=300)

alt.Chart(...)

# Line chart

In [ ]:
base = alt.Chart(nbr_df).encode(
    x=alt.X('Timestamp:T', title='Date'))

line = base.mark_line().encode(
    y=alt.Y('median(NDVI):Q', scale=alt.Scale(domain=(0.0, 1))))

band = base.mark_errorband(extent='iqr').encode(
    y='NDVI:Q')

(line + band).properties(width=600, height=300).interactive()

alt.LayerChart(...)